In [1]:
import sqlalchemy as db
import pandas as pd
import numpy as np
import json
import operator

from utilidades import conectar_db
from utilidades import reducir_dimensionalidad_dataset
from sklearn.metrics.pairwise import cosine_similarity

connection = conectar_db()
df_reducido = reducir_dimensionalidad_dataset()

[pca] >Processing dataframe..
[pca] >Normalizing input data per feature (zero mean and unit variance)..
[pca] >The PCA reduction is performed on the [67] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Outlier detection using Hotelling T2 test with alpha=[0.05] and n_components=[11]
[pca] >Multiple test correction applied for Hotelling T2 test: [fdr_bh]
[pca] >Outlier detection using SPE/DmodX with n_std=[3]
[pca] >Processing dataframe..
[pca] >Normalizing input data per feature (zero mean and unit variance)..
[pca] >Outlier detection using Hotelling T2 test with alpha=[0.05] and n_components=[11]
[pca] >Multiple test correction applied for Hotelling T2 test: [fdr_bh]
[pca] >Outlier detection using SPE/DmodX with n_std=[3]


In [2]:
def obtener_posicion_segun_jugador(jugador_id):
    return pd.read_sql("SELECT posicion from Jugadores where jugador_id = " + str(jugador_id), con=connection)['posicion'][0]
    
def obtener_dataset_segun_posicion(posicion_jugador, dataset):
    return dataset[dataset["posicion"] == posicion_jugador]

def obtener_estadisticas_por_jugador(id_jugador):
    jugador_elegido = df_reducido[df_reducido.iloc[:,0] == id_jugador]
    array_estadisticas = np.array(jugador_elegido.iloc[:,2:])
    return array_estadisticas


def obtener_jugadores_similares(id_jugador, n_similares):
    estadisticas_jugador_elegido = obtener_estadisticas_por_jugador(id_jugador)
    df_filtrado_sin_jugador_elegido = df_reducido.loc[df_reducido['jugador_id'] != id_jugador]
    dicc_jugador_relacion = dict()
    
    for i in df_filtrado_sin_jugador_elegido.index:
        id_jugador_iterado = df_filtrado_sin_jugador_elegido['jugador_id'][i]
        estadisticas_jugador = obtener_estadisticas_por_jugador(id_jugador_iterado)
        porcentaje_relacion = cosine_similarity(estadisticas_jugador_elegido,estadisticas_jugador)[0][0]
        nombre_jugador = pd.read_sql("SELECT nombre from Jugadores where jugador_id = " + str(id_jugador_iterado), con=connection)['nombre'][0]
        dicc_jugador_relacion[nombre_jugador] = porcentaje_relacion

    
    dicc_jugador_relacion_ordenado = dict(sorted(dicc_jugador_relacion.items(), key=operator.itemgetter(1), reverse=True))
    dicc_jugador_relacion_filtrado = dict(list(dicc_jugador_relacion_ordenado.items())[0: n_similares])
    return json.dumps(dicc_jugador_relacion_filtrado)

def mostrar_jugadores_similares():
    nombre = input("Ingrese el nombre del jugador: ")
    try:
        id_jugador = pd.read_sql_query("select jugador_id from Jugadores where nombre='" + nombre + "'", con=connection)['jugador_id'][0]
        print("Los jugadores mas similares a "+ nombre + " son: ")
        return obtener_jugadores_similares(id_jugador, 15)

    except:
        print("El jugador ingresado (" + nombre + ") no existe en la base de datos")

mostrar_jugadores_similares()



Los jugadores mas similares a Julian Alvarez son: 


'{"Martin Ojeda": 0.9831031630522477, "Diego Valeri": 0.9332509762918078, "Lucas Janson": 0.9128643968905823, "Matias Suarez": 0.9101692349427387, "Cristian Pavon": 0.9007095060456934, "Lucas Gamba": 0.9002575252778251, "Gabriel Hauche": 0.899609281771773, "Francisco Pizzini": 0.8995273460786928, "Jonas Acevedo": 0.8708057875552374, "Pablo Mouche": 0.8674211652164046, "Francisco Gonzalez Metilli": 0.8581438295723998, "Jesus Datolo": 0.8570212177591889, "Tomas Chancalay": 0.856417086423628, "Luis Rodriguez": 0.8549594789393474, "Leandro Fernandez": 0.8537418481712836}'